<a href="https://colab.research.google.com/github/itsokaypiyush/Text-to-speech-/blob/main/script_to_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


from google.colab import files
import os
import torch
from transformers import pipeline
import whisper
from elevenlabs import ElevenLabs
import pypdf
from moviepy.editor import *
import subprocess


# =============================================================
#          STEP 1 — UPLOAD PDF, VIDEO, MUSIC
# =============================================================

print(" Please upload your PDF first:")
pdf_upload = files.upload()
pdf_name = list(pdf_upload.keys())[0]

print("\n Please upload your background video:")
video_upload = files.upload()
video_name = list(video_upload.keys())[0]

print("\n Upload background music (optional). If you skip, press cancel:")
try:
    music_upload = files.upload()
    music_name = list(music_upload.keys())[0]
except:
    music_name = None
    print("⏭ No music uploaded. Moving ahead...")


# =============================================================
#          STEP 2 — Ask API Key + Voice ID
# =============================================================

ELEVEN_API_KEY = input(" Enter your ElevenLabs API Key: ").strip()
VOICE_ID = input(" Enter your ElevenLabs Voice ID: ").strip()


# =============================================================
#          STEP 3 — Extract PDF text
# =============================================================

print("\n📄 Extracting PDF text...")

def extract_pdf_text(path):
    txt = ""
    for page in pypdf.PdfReader(path).pages:
        content = page.extract_text()
        if content:
            txt += content + "\n"
    return txt

full_text = extract_pdf_text(pdf_name)

print("✔ PDF text extracted!\n")


# =============================================================
#          STEP 4 — Summarize using BART
# =============================================================

print(" Summarizing text using HuggingFace BART...")

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary = summarizer(
    full_text,
    max_length=250,
    min_length=80,
    do_sample=False
)[0]['summary_text']

print("\n✔ Summary Ready:")
print(summary, "\n")


# =============================================================
#          STEP 5 — ElevenLabs TTS
# =============================================================

print("🎙 Generating voice with ElevenLabs...")

client = ElevenLabs(api_key=ELEVEN_API_KEY)

audio_stream = client.text_to_speech.convert(
    voice_id=VOICE_ID,
    model_id="eleven_multilingual_v2",
    text=summary,
    output_format="mp3_44100_128",
)

TTS_AUDIO = "voiceover.mp3"

with open(TTS_AUDIO, "wb") as f:
    for chunk in audio_stream:
        f.write(chunk)

print("✔ Voiceover created!\n")


# =============================================================
#          STEP 6 — Whisper to generate captions
# =============================================================

print("📝 Running Whisper to create subtitles (SRT)...")

whisper_model = whisper.load_model("small")
result = whisper_model.transcribe(TTS_AUDIO)

SRT_FILE = "captions.srt"

def write_srt(result, filename):
    def ts(t):
        h = int(t // 3600)
        m = int((t % 3600) // 60)
        s = t % 60
        return f"{h:02}:{m:02}:{s:06.3f}".replace('.', ',')

    with open(filename, "w", encoding="utf-8") as f:
        for i, seg in enumerate(result["segments"], 1):
            f.write(f"{i}\n")
            f.write(f"{ts(seg['start'])} --> {ts(seg['end'])}\n")
            f.write(seg['text'].strip() + "\n\n")

write_srt(result, SRT_FILE)

print("✔ Subtitles generated:", SRT_FILE, "\n")


# =============================================================
#          STEP 7 — Mix background music
# =============================================================

print("🎶 Mixing background music...")

if music_name:
    voice = AudioFileClip(TTS_AUDIO)
    music = AudioFileClip(music_name).volumex(0.15)

    MIXED_AUDIO = "mixed_audio.mp3"
    final_audio = CompositeAudioClip([music, voice])
    final_audio.write_audiofile(MIXED_AUDIO)
else:
    MIXED_AUDIO = TTS_AUDIO

print("✔ Audio is ready!\n")


# =============================================================
#          STEP 8 — Merge Audio + Video
# =============================================================

print("🎬 Merging narration and video...")

video = VideoFileClip(video_name)
audio = AudioFileClip(MIXED_AUDIO)

TEMP_VIDEO = "temp_video.mp4"
final_base = video.set_audio(audio)
final_base.write_videofile(TEMP_VIDEO, fps=30)

print("✔ Video with voice done!\n")


# =============================================================
#          STEP 9 — Burn Subtitles (FFmpeg)
# =============================================================

print("🔥 Burning subtitles into video...")

FINAL_OUTPUT = "final_subtitled_video.mp4"

subprocess.run([
    "ffmpeg",
    "-i", TEMP_VIDEO,
    "-vf", f"subtitles={SRT_FILE}",
    "-c:a", "copy",
    FINAL_OUTPUT
])

print("\n🎉 FINAL VIDEO READY:")
print("📽 Output File:", FINAL_OUTPUT)

files.download(FINAL_OUTPUT)


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



📤 Please upload your PDF first:


Saving funny_dark_school_script.pdf to funny_dark_school_script.pdf

🎥 Please upload your background video:


Saving Subway Surfer Highscore 10512290 , World Record !.mp4 to Subway Surfer Highscore 10512290 , World Record !.mp4

🎶 Upload background music (optional). If you skip, press cancel:


⏭ No music uploaded. Moving ahead...
🔑 Enter your ElevenLabs API Key: sk_abd9e2076933d3b406effb83d3ed71661c015404ff4d7937
🎤 Enter your ElevenLabs Voice ID: Sm1seazb4gs7RSlUVw7c

📄 Extracting PDF text...
✔ PDF text extracted!

🧠 Summarizing text using HuggingFace BART...


...

MoviePy - Done.
Moviepy - Writing video temp_video.mp4



t: 100%|█████████▉| 10489/10493 [16:32<00:00, 30.30it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file Subway Surfer Highscore 10512290 , World Record !.mp4, 2764800 bytes wanted but 0 bytes read,at frame 8742/8745, at time 349.68/349.76 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|██████████| 10493/10493 [16:32<00:00, 28.09it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file Subway Surfer Highscore 10512290 , World Record !.mp4, 2764800 bytes wanted but 0 bytes read,at frame 8743/8745, at time 349.72/349.76 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready temp_video.mp4
✔ Video with voice done!

🔥 Burning subtitles into video...

🎉 FINAL VIDEO READY:
📽 Output File: final_subtitled_video.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>